# 🚀 GENERATE ASSETS (OG)
-------

In [36]:
# import nltk

# # Download required NLTK data (only need to do this once)
# nltk.download('punkt')

In [37]:
import trendspy
from trendspy import Trends
import os
import datetime
from dotenv import load_dotenv
from news_utils import News
import feedparser
import time
import pickle
from generateAssets import *
load_dotenv()

True

In [38]:
# Make directory for the run
APPDIR = os.getenv("APPDIR")
BASEPATH = os.getenv("BASEPATH")
RUN_DIR = os.path.join(BASEPATH, datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(BASEPATH, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.chdir(APPDIR)

In [39]:
os.environ["RUN_DIR"] = RUN_DIR

In [44]:
def getTrends():
    def freeze(d : dict):
        return 
    tr = Trends()
    trends = tr.trending_now(geo='US', hours=24)
    businessFiltered = trends.filter_by_topic('Business and Finance')
    techFiltered = trends.filter_by_topic('Technology')
    lawAndGovFiltered = trends.filter_by_topic('Law and Government')
    politicsFiltered = trends.filter_by_topic('Politics')
    filteredTrends = businessFiltered + techFiltered + lawAndGovFiltered + politicsFiltered
    sortedTrends = sorted(filteredTrends, key=lambda x: x.volume, reverse=True)
    keysSoFar = set()
    finalTrendList = []
    for trend in sortedTrends:
        if not trend.keyword in keysSoFar:
            finalTrendList.append({
                'keyword': trend.keyword,
                'volume': trend.volume,
                'volume_growth_pct': trend.volume_growth_pct,
                'topics': trend.topics,
                'keywords': trend.trend_keywords,
            })
            keysSoFar.add(trend.keyword)
    return finalTrendList

In [45]:
# Example usage in your getNews function:
def getNews(trendKeyword, num_articles=10, maxAge=10, linkNews={}):
    #maxAge is the maximum age of the news article in days
    search_term = trendKeyword.replace(' ', '+')
    url = f'https://news.google.com/rss/search?q={search_term}&hl=en-US&gl=US&ceid=US:en'
    feed = feedparser.parse(url)
    valid, invalid = [], []
    i = 0
    while i < len(feed.entries) and len(valid) < num_articles:
        entry = feed.entries[i]
        i += 1
        if entry.link not in linkNews:
            pub_date = datetime.datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %Z')
            if datetime.datetime.now() - pub_date > datetime.timedelta(days=maxAge):
                continue

            newNews = News(
                entry.title,
                entry.link,
                entry.published,
            )
            linkNews[entry.link] = newNews
            if newNews.mainText:
                valid.append(newNews)
            else:
                invalid.append(newNews)

    return valid, invalid

In [2]:
def collectArticles(trend, linkNews={}, maxArticles = 30):
    news = set()
    keywords = set(trend['keywords'] + [trend['keyword']])
    numArticlesPerKeyword = 1 + (maxArticles // len(keywords)) # promotes diversity across keywords
    for keyword in keywords:
        time.sleep(2) # wait two seconds to avoid rate limiting
        valid, invalid = getNews(keyword, num_articles=numArticlesPerKeyword, linkNews=linkNews)
        news.update(valid)
        if len(news) > maxArticles:
            break
    trend['news'] = list(news)

In [1]:
def trendTemplate():
    return {'keyword': '', 'volume': -1, 'volume_growth_pct': -1, 'topics': [], 'keywords': []}

# 🔄 Collect Trends Or Load From Pickle
---

In [47]:
getTrendInfo = input('Get Trends? (y/n): ')
if getTrendInfo.lower() == 'y':
    finalTrendList = getTrends()
    showTrends = input('Show Trends? (y/n): ')
    if showTrends.lower() == 'y':
        # outStr = """"""
        for trend in finalTrendList[:min(100, len(finalTrendList))]:
            print(trend)
    saveTrends = input('Save Trends? (y/n): ')
    if saveTrends.lower() == 'y':
        with open('./tmp/finalTrendList.pkl', 'wb') as f:
            pickle.dump(finalTrendList, f)
elif getTrendInfo.lower() == 'n':
    with open('./tmp/finalTrendList.pkl', 'rb') as f:
        finalTrendList = pickle.load(f)
else:
    print('Invalid input, run again')
    exit()

{'keyword': 'guinness', 'volume': 200000, 'volume_growth_pct': 100, 'topics': [10, 8], 'keywords': ['guinness', 'who is st patrick', 'shamrock', "st patrick's day 2025", 'who was st patrick', 'st. patrick', 'saint patricks day', 'st patricks', 'st pattys day', 'st patricks day 2025', 'what is st patricks day', "st. patrick's day 2025", 'who is saint patrick', 'saint-patrick', "st patrick's day parade 2025", 'saint patrick day', 'st paddys day', 'who was saint patrick', 'st patrick day 2025', "st.patrick's day"]}
{'keyword': 'forever 21', 'volume': 100000, 'volume_growth_pct': 600, 'topics': [3], 'keywords': ['forever 21', 'forever 21 closing', 'forever21', 'shein', 'mall']}
{'keyword': 'autopen', 'volume': 100000, 'volume_growth_pct': 1000, 'topics': [14, 10], 'keywords': ['autopen', 'what is autopen', 'auto pen', 'biden pardons', 'biden autopen', 'autopen machine', 'adam kinzinger', 'autopen signature', 'trump biden pardons', 'trump pardons', 'pardons', 'joe biden']}
{'keyword': 'cono

# 🎯 Select Trend and Collect Articles
---

In [48]:
keyword = input('Enter trend keyword: ')
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]
linkNews = {}
collectArticles(trend, linkNews=linkNews, maxArticles = 40)
pickle.dump(finalTrendList, open('./tmp/finalTrendList.pkl', 'wb'))
print('Trends info scraped, trend selected and articles collected. All information saved.')

URL https://news.google.com/rss/articles/CBMiWkFVX3lxTE4xSng5R2FiTGJYMDRuT3Jmd0ZtSFAxay1LV1RWRHQ1dnBSRE5FcWxNREFESFV1NjBvZlB2WGlZMVJnVTBEdjJQTUtBbmRZNElNMExOajVkeDc0UdIBX0FVX3lxTE95cEV4VHoxSklETUlTbXlXRHZBMzlKU0FBSy00dVNTWHlzOHZsUVg2azZLdWVmZXpsbEJPdHZnRC1JWGVDVXZGc0t5N0NuMUFrZHdsT3ZRNXp3cjhXTzkw?oc=5
URL https://news.google.com/rss/articles/CBMinwFBVV95cUxPZU9qYno1VnJta0plZWNKWG5ES1JFZHpNQ3lPUnV0X19IeFdZLVdRYUhhLVNmUmx5MTE1VVYyc2xoUUNEcEtZN2VOMHhHNmw3a0xNTi1lMUtOSm5RTFk0Wk85T2YwR290a1B4WC13UWZUUEM5eUx2aXc1emtEc3FZVmlOU1ZJYmt0dWZwTlZJN1RXRUtqYno4QkktMWVLdG_SAXNBVV95cUxPVk5BV3N6c0U1TlZFc0N4RFI5MDQtOFBGVkNsS0hsSzlwMU91UXUwNVQ4UWJHZXJhRjM2VUZqek5udmVSeXA3RUJ6OXM4QmZIYTVhYmVhQXpiMmRYTXhmV3FKZjlOQmlISExFQk1ZeXZWWkJz?oc=5
URL https://news.google.com/rss/articles/CBMieEFVX3lxTFA3Q2UxVXZVTDdQc08xMHY3VF84cnpwRkJwRFdNWUoxd25iX2ZNSGxUN3NtSGpVQ1c5NWlWNTZBN2xQeTFxS1V1NUtkZzgzdHpmWUlydnV2dmVuMUY5UW1Nb3liWTJnLUtmWHpRMjE1UjRBM0FNZlFIYQ?oc=5
Error extracting article content: Article `download()` failed

# 🎨 Generate Assets For Selected Trend
---

In [49]:
config = {'news_report': True,
        'script': True,
        'narration': True,
        'voice': False,
        'sora_prompts': True}

In [50]:
print(ELEVENLABS_API_KEY)

sk_f96d619fc51235180796190a025b96954210e5b25af0d67d


In [51]:
generate_assets(trend, openai_api_key=OPENAI_API_KEY, elevenlabs_api_key=ELEVENLABS_API_KEY, voice_id=VOICE_ID, run_dir=RUN_DIR, config=config)

C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-17-19-22-52\autopen topic_dir
C:\Users\garre\OneDrive\Desktop\Projects\aiMedia\assets\2025-03-17-19-22-52\autopen\sora_prompts sora_prompt_dir
20 validArticles


{'paths': {'script': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52\\autopen\\script.txt',
  'audio': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52\\autopen\\voiceover.mp3',
  'video': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52\\autopen\\video.mp4',
  'sora_prompts': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52\\autopen\\sora_prompts',
  'news_report': 'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52\\autopen\\news_report.txt'}}

In [52]:
os.getenv("RUN_DIR")

'C:\\Users\\garre\\OneDrive\\Desktop\\Projects\\aiMedia\\assets\\2025-03-17-19-22-52'

In [53]:
os.chdir(RUN_DIR)
outName = trend['keyword'].replace(" ", "_") # name of folder and file to zip
!tar -acf {outName}.zip {outName}

In [54]:
!echo python -m wormhole send {outName}.zip | clip && start cmd